<p style="align: center;"><img align=center src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500 height=450/></p>

<h3 style="text-align: center;"><b>Школа глубокого обучения ФПМИ МФТИ</b></h3>

<h3 style="text-align: center;"><b>Домашнее задание. Продвинутый поток. Весна 2021</b></h3>

Это домашнее задание будет посвящено полноценному решению задачи машинного обучения.

Есть две части этого домашнего задания: 
* Сделать полноценный отчет о вашей работе: как вы обработали данные, какие модели попробовали и какие результаты получились (максимум 10 баллов). За каждую выполненную часть будет начислено определенное количество баллов.
* Лучшее решение отправить в соревнование на [kaggle](https://www.kaggle.com/c/advanced-dls-spring-2021/) (максимум 5 баллов). За прохождение определенного порогов будут начисляться баллы.


**Обе части будут проверяться в формате peer-review. Т.е. вашу посылку на степик будут проверять несколько других студентов и аггрегация их оценок будет выставлена. В то же время вам тоже нужно будет проверить несколько других учеников.**

**Пожалуйста, делайте свою работу чистой и понятной, чтобы облегчить проверку. Если у вас будут проблемы с решением или хочется совета, то пишите в наш чат в телеграме или в лс @runfme. Если вы захотите проаппелировать оценку, то пипшите в лс @runfme.**

**Во всех пунктах указания это минимальный набор вещей, которые стоит сделать. Если вы можете сделать какой-то шаг лучше или добавить что-то свое - дерзайте!**

# Как проверять?

Ставьте полный балл, если выполнены все рекомендации или сделано что-то более интересное и сложное. За каждый отсустствующий пункт из рекомендация снижайте 1 балл.

# Метрика

Перед решением любой задачи важно понимать, как будет оцениваться ваше решение. В данном случае мы используем стандартную для задачи классификации метрику ROC-AUC. Ее можно вычислить используя только предсказанные вероятности и истинные классы без конкретного порога классификации + она раотает даже если классы в данных сильно несбалансированны (примеров одного класса в десятки раз больше примеров длугого). Именно поэтому она очень удобна для соревнований.

Посчитать ее легко:


# Первая часть. Исследование

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

from category_encoders import OrdinalEncoder, OneHotEncoder
from category_encoders import BackwardDifferenceEncoder, HelmertEncoder, BinaryEncoder
from category_encoders import CountEncoder
from category_encoders import LeaveOneOutEncoder, TargetEncoder, JamesSteinEncoder, MEstimateEncoder, WOEEncoder, CatBoostEncoder
from category_encoders.wrapper import NestedCVWrapper

## Загрузка данных (2 балла)

1) Посмотрите на случайные строчки. 

2) Посмотрите, есть ли в датасете незаполненные значения (nan'ы) с помощью data.isna() или data.info() и, если нужно, замените их на что-то. Будет хорошо, если вы построите табличку с количеством nan в каждой колонке.

In [17]:
data = pd.read_csv('./train.csv')

In [18]:
# Для вашего удобства списки с именами разных колонок

# Числовые признаки
num_cols = [
    'ClientPeriod',
    'MonthlySpending',
    'TotalSpent'
]

# Категориальные признаки
cat_cols = [
    'Sex',
    'IsSeniorCitizen',
    'HasPartner',
    'HasChild',
    'HasPhoneService',
    'HasMultiplePhoneNumbers',
    'HasInternetService',
    'HasOnlineSecurityService',
    'HasOnlineBackup',
    'HasDeviceProtection',
    'HasTechSupportAccess',
    'HasOnlineTV',
    'HasMovieSubscription',
    'HasContractPhone',
    'IsBillingPaperless',
    'PaymentMethod'
]

feature_cols = num_cols + cat_cols
target_col = 'Churn'

In [19]:
data.head()

,ClientPeriod,MonthlySpending,TotalSpent,Sex,IsSeniorCitizen,HasPartner,HasChild,HasPhoneService,HasMultiplePhoneNumbers,HasInternetService,HasOnlineSecurityService,HasOnlineBackup,HasDeviceProtection,HasTechSupportAccess,HasOnlineTV,HasMovieSubscription,HasContractPhone,IsBillingPaperless,PaymentMethod,Churn
0,55,19.50,1026.35,Male,0,Yes,Yes,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,No,Mailed check,0
1,72,25.85,1872.2,Male,0,Yes,No,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Credit card (automatic),0
2,1,75.90,75.9,Male,0,No,No,Yes,No,Fiber optic,No,No,No,Yes,No,No,Month-to-month,Yes,Electronic check,1
3,32,79.30,2570,Female,1,Yes,No,Yes,Yes,Fiber optic,No,No,Yes,No,No,No,Month-to-month,No,Mailed check,0
4,60,115.25,6758.45,Female,0,Yes,Yes,Yes,Yes,Fiber optic,Yes,Yes,Yes,Yes,Yes,Yes,Two year,No,Credit card (automatic),0


In [ ]:
data.info()

### Первичные выводы 
- Пропусков нет
- 5282 строк
- Из аномалий - категориальный признак IsSeniorCitizen представлен в числовом виде, в принципе подходит для one-hot encoding
- TotalSpent строковый, хотя должны быть числа
- Осталось посмотреть есть статистику по классам, какие есть градации по качественным признакам, статистику по количественным и аномалии

## Анализ данных (3 балла)

1) Для численных призанков постройте гистограмму (*plt.hist(...)*) или boxplot (*plt.boxplot(...)*). Для категориальных посчитайте количество каждого значения для каждого признака. Для каждой колонки надо сделать *data.value_counts()* и построить bar диаграммы *plt.bar(...)* или круговые диаграммы *plt.pie(...)* (хорошо, елси вы сможете это сделать на одном гарфике с помощью *plt.subplots(...)*). 

2) Посмотрите на распределение целевой переменной и скажите, являются ли классы несбалансированными.

3) (Если будет желание) Поиграйте с разными библиотеками для визуализации - *sns*, *pandas_visual_analysis*, etc.

Второй пункт очень важен, потому что существуют задачи классификации с несбалансированными классами. Например, это может значить, что в датасете намного больше примеров 0 класса. В таких случаях нужно 1) не использовать accuracy как метрику 2) использовать методы борьбы с imbalanced dataset (обычно если датасет сильно несбалансирован, т.е. класса 1 в 20 раз меньше класса 0).

In [ ]:
# посмотрим на количественные признаки
data.describe()

In [ ]:
# ClienPerion есть нули, что это может значить?
print(len(data[data['ClientPeriod'] == 0]))
data[data['ClientPeriod'] == 0].head()

In [ ]:
fig = px.histogram(
    data,
    x="MonthlySpending",
    width=600,
    height=300,
    title="Гистограмма по месячным затратам",
    labels={"x": "Затраты в месяц"},
)
fig.update_yaxes(title_text="количество")
fig.show()

In [ ]:
#преобразуем TotalSpent в числа и посмотрим на данные
data['TotalSpent'] = pd.to_numeric(data['TotalSpent'], errors='coerce')

In [ ]:
data['TotalSpent'].isna().sum()

In [ ]:
data['TotalSpent'].describe()

In [ ]:
# посмотрим на категориальные признаки
for column in cat_cols:
    print(column, data[column].unique())

### Рекомендации по подготовке данных
1. ClientPeriod, нулевые их всего 9 штук, непонятно, что может значть 0 в этой колонке, когда MontlySpending есть. У них отсутствует TotalSpent, так что это возможно те, которые только подключились. Если мы их удалим, то возможно, модель не научится работать с такими абонентами, попробуем оставить. У них пропуски в TotalSpent надо заменить на нули
2. 26% от общего числа принадлежат к классу 1. В общем классы несбалансированы, но не радикально
3. Категориальные признаки понятные, 2-4 градации на признак

(Дополнительно) Если вы нашли какие-то ошибки в данных или выбросы, то можете их убрать. Тут можно поэксперементировать с обработкой данных как угодно, но не за баллы.

## Применение линейных моделей (3 балла)

1) Обработайте данные для того, чтобы к ним можно было применить LogisticRegression. Т.е. отнормируйте числовые признаки, а категориальные закодируйте с помощью one-hot-encoding'а. 

2) С помощью кроссвалидации или разделения на train/valid выборку протестируйте разные значения гиперпараметра C и выберите лучший (можно тестировать С=100, 10, 1, 0.1, 0.01, 0.001) по метрике ROC-AUC. 

Если вы разделяете на train/valid, то используйте LogisticRegressionCV. Он сам при вызове .fit() подберет параметр С. (не забудьте передать scroing='roc_auc', чтобы при кроссвалидации сравнивались значения этой метрики, и refit=True, чтобы при потом модель обучилась на всем датасете с лучшим параметром C). 


(более сложный вариант) Если вы будете использовать кроссвалидацию, то преобразования данных и LogisticRegression нужно соединить в один Pipeline с помощью make_pipeline, как это делалось во втором семинаре. Потом pipeline надо передать в GridSearchCV. Для one-hot-encoding'a можно испльзовать комбинацию LabelEncoder + OneHotEncoder (сначала превращаем строчки в числа, а потом числа првращаем в one-hot вектора.)

In [5]:
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, RobustScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [6]:
# Напишем функцию для чистки данных (это можно делать до разделения на train|val)
def data_cleaning(data):
    if data['TotalSpent'].dtype != 'float64':
        data['TotalSpent'] = pd.to_numeric(data['TotalSpent'], errors='coerce')
    
    #if data['IsSeniorCitizen'].dtype == 'int64':
     #   data['IsSeniorCitizen'] = data['IsSeniorCitizen'].astype(str)
        
    data.fillna(value = 0, inplace=True)

In [20]:
data_cleaning(data)

In [21]:
data.head(2)

,ClientPeriod,MonthlySpending,TotalSpent,Sex,IsSeniorCitizen,HasPartner,HasChild,HasPhoneService,HasMultiplePhoneNumbers,HasInternetService,HasOnlineSecurityService,HasOnlineBackup,HasDeviceProtection,HasTechSupportAccess,HasOnlineTV,HasMovieSubscription,HasContractPhone,IsBillingPaperless,PaymentMethod,Churn
0,55,19.50,1026.35,Male,0,Yes,Yes,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,No,Mailed check,0
1,72,25.85,1872.20,Male,0,Yes,No,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Credit card (automatic),0


## Alex AutoML

In [9]:
import automl_alex
import time
from automl_alex import DataPrepare
from automl_alex import AutoML, AutoMLClassifier, AutoMLRegressor
print('AutoML-Alex version:', automl_alex.__version__)

AutoML-Alex version: 1.3.10


In [22]:
# кажется, здесь нам это не нужно, можно обучить на всем сете и затем проверить на тесте
data_train, data_val, target_train, target_val = train_test_split(data[feature_cols], data[target_col], 
                                                    train_size=0.8,
                                                    random_state=42)

IndentationError: unexpected indent (<ipython-input-22-a279657040c6>, line 2)

In [24]:
model = AutoMLClassifier(random_state=42)
model.fit(data[feature_cols], data[target_col], timeout=2000)

11:31:02 | > Start Fit Base Model
11:31:46 | ##################################################
11:31:46 | > Start Fit Models 2
11:31:46 | ##################################################
11:31:46 | ##################################################
11:31:47 | > Step 1: calc parameters and pruned score: get test 10 trials
11:32:53 |  One iteration ~ 6.6 sec
11:32:53 |  Possible iters ~ 276.0
11:32:53 | --------------------------------------------------
11:32:53 |   Pruned Threshold Score: 0.8284
11:32:53 | ##################################################
11:32:53 | > Step 2: Full opt with Threshold Score Pruner
11:32:53 | ##################################################
11:32:53 | > Start optimization with the parameters:
11:32:53 | CV_Folds = 7
11:32:53 | Score_CV_Folds = 3
11:32:53 | Feature_Selection = False
11:32:53 | Opt_lvl = 2
11:32:53 | Cold_start = 20
11:32:53 | Early_stoping = 30
11:32:53 | Metric = roc_auc_score
11:32:53 | Direction = maximize
11:32:53 | ##############

In [57]:
X_test = pd.read_csv('./test.csv')
submission = pd.read_csv('./submission.csv')

In [58]:
data_cleaning(X_test)

In [27]:
predicts = model.predict(X_test)
predicts

0       0.031769
1       0.672322
2       0.620734
3       0.017940
4       0.005140
          ...   
1756    0.126961
1757    0.012736
1758    0.796514
1759    0.520109
1760    0.012345
Length: 1761, dtype: float64

In [28]:
submission['Churn'] = predicts
submission.to_csv('./my_submission.csv', index=False)

0.85339 - это лучший мой результат! И он получен на AutoML, есть повод задуматься. Это результат получен на неполной выборке
На полном датасете он показал еще больше 0.85469

### А что есть доверить Alex и чистку данных?

In [29]:
from automl_alex import DataPrepare

In [30]:
X_train, y_train = data[feature_cols], data[target_col]

In [33]:
%%time
de = DataPrepare(clean_and_encod_data=True,
                cat_encoder_names=['HelmertEncoder','OneHotEncoder'], # Encoders list for Generator cat encodet features
                clean_nan=True, # fillnan
                clean_outliers=True, # method='IQR', threshold=2,
                drop_invariant=True, # drop invariant features (data.nunique < 2)
                num_generator_features=True, # Generator interaction Num Features
                num_denoising_autoencoder=True, # denoising_autoencoder if num features > 2
                normalization=True, # normalization data (StandardScaler)
                cat_features=None, # DataPrepare can auto detect categorical features
                random_state=42,
                verbose=3)
clean_X_train = de.fit_transform(X_train)

11:58:15 | Source data shape: (5282, 19)
11:58:15 | ##################################################
11:58:15 | ! START preprocessing Data
11:58:15 | - Auto detect cat features: 10
11:58:15 | > Binary Features
11:58:15 | > Clean Categorical Features
11:58:15 | > Transform Categorical Features.
11:58:15 |  - Encoder: HelmertEncoder ADD features: 21
11:58:16 |  - Encoder: OneHotEncoder ADD features: 31
11:58:16 |   No nans features
11:58:16 | > Start fit DenoisingAutoencoder


Epoch 1/50
Epoch 2/50
Epoch 3/50
Epoch 4/50
Epoch 5/50
Epoch 6/50
Epoch 7/50
Epoch 8/50
Epoch 9/50
Epoch 10/50
Epoch 11/50
Epoch 12/50
Epoch 13/50
Epoch 14/50
Epoch 15/50
Epoch 16/50
Epoch 17/50
Epoch 18/50
Epoch 19/50
Epoch 20/50
Epoch 21/50
Epoch 22/50
Epoch 23/50
Epoch 24/50
Epoch 25/50
Epoch 26/50
Epoch 27/50
Epoch 28/50
Epoch 29/50
Epoch 30/50
Epoch 31/50
Epoch 32/50
Epoch 33/50
Epoch 34/50
Epoch 35/50
Epoch 36/50
Epoch 37/50
Epoch 38/50
Epoch 39/50
Epoch 40/50
Epoch 41/50
Epoch 42/50
Epoch 43/50
Epoch 44/50
Epoch 45/50
Epoch 46/50
Epoch 47/50
Epoch 48/50
Epoch 49/50
Epoch 50/50


11:59:02 | > Add Denoising features
11:59:02 | > CleanOutliers
11:59:02 | > Generate interaction Num Features
11:59:02 |  ADD features: 15
11:59:02 | > Normalization Features
11:59:02 | ##################################################
11:59:02 | Final data shape: (5282, 92)
11:59:02 | Total ADD columns: 73
11:59:02 | Time fit data_preproc: 47.06 sec
11:59:02 | ##################################################


Wall time: 47.1 s


In [38]:
clean_X_train.head(5)

,Sex,IsSeniorCitizen,HasPartner,HasChild,HasPhoneService,IsBillingPaperless,HelmertEncoder_HasMultiplePhoneNumbers_1,HelmertEncoder_HasInternetService_1,HelmertEncoder_HasOnlineSecurityService_1,HelmertEncoder_HasOnlineBackup_1,...,TotalSpent_-_ClientPeriod,TotalSpent_/_MonthlySpending,TotalSpent_-_MonthlySpending,MonthlySpending_/_ClientPeriod,MonthlySpending_-_ClientPeriod,MonthlySpending_/_TotalSpent,MonthlySpending_-_TotalSpent,ClientPeriod_*_TotalSpent,ClientPeriod_*_MonthlySpending,TotalSpent_*_MonthlySpending
0,1,0,1,1,1,1,-1.0,-1.0,-1.0,-1.0,...,-0.570910,0.823215,-0.540490,-0.506536,-2.011662,-0.490326,0.540490,-0.406050,-0.535987,-0.727311
1,1,0,1,0,1,1,-1.0,-1.0,-1.0,-1.0,...,-0.202300,1.628250,-0.167259,-0.506259,-2.326595,-0.508858,0.167259,0.093384,-0.188233,-0.608082
2,1,0,0,0,1,0,-1.0,-1.0,-1.0,-1.0,...,-0.969583,-1.276936,-0.988123,4.157874,1.252998,3.011050,0.988123,-0.765399,-0.975409,-0.787185
3,0,1,1,0,1,1,-1.0,-1.0,-1.0,-1.0,...,0.125817,0.000587,0.119211,-0.375420,0.436833,-0.448008,-0.119211,-0.241647,0.110006,0.044740
4,0,0,1,1,1,1,-1.0,-1.0,2.0,2.0,...,1.976071,1.067599,1.965360,-0.409829,0.671924,-0.497274,-1.965360,1.819004,2.040092,2.460658


In [59]:
X_train = X_test[feature_cols]

In [60]:
clean_X_test = de.transform(X_test)

13:14:09 | ##################################################
13:14:09 | ! Start Transform Data
13:14:09 | > Clean Binary Features
13:14:09 | > Clean Categorical Features
13:14:09 | > Transform Categorical Features.
13:14:09 |  - Encoder: HelmertEncoder ADD features: 21
13:14:09 |  - Encoder: OneHotEncoder ADD features: 31
13:14:10 | > Add Denoising features
13:14:10 | > Generate interaction Num Features
13:14:10 |  ADD features: 15
13:14:10 | > Normalization Features
13:14:10 | ##################################################
13:14:10 | Final data shape: (1761, 92)
13:14:10 | Total ADD columns: 73
13:14:10 | ##################################################


In [36]:
clean_X_test.head(5)

,Sex,IsSeniorCitizen,HasPartner,HasChild,HasPhoneService,IsBillingPaperless,HelmertEncoder_HasMultiplePhoneNumbers_1,HelmertEncoder_HasInternetService_1,HelmertEncoder_HasOnlineSecurityService_1,HelmertEncoder_HasOnlineBackup_1,...,TotalSpent_-_ClientPeriod,TotalSpent_/_MonthlySpending,TotalSpent_-_MonthlySpending,MonthlySpending_/_ClientPeriod,MonthlySpending_-_ClientPeriod,MonthlySpending_/_TotalSpent,MonthlySpending_-_TotalSpent,ClientPeriod_*_TotalSpent,ClientPeriod_*_MonthlySpending,TotalSpent_*_MonthlySpending
0,1,0,1,0,1,1,-1.0,2.0,2.0,-1.0,...,0.039919,0.412934,0.048099,-0.445956,-0.544931,-0.474249,-0.048099,-0.126858,0.030023,-0.248890
1,1,1,1,0,1,0,-1.0,-1.0,-1.0,-1.0,...,0.320229,-0.077085,0.303692,-0.318713,1.093314,-0.441112,-0.303692,-0.210538,0.250615,0.431665
2,1,0,0,0,1,0,-1.0,-1.0,-1.0,-1.0,...,-0.702326,-0.952288,-0.717547,-0.005326,1.026779,-0.160751,0.717547,-0.726593,-0.706293,-0.592021
3,0,0,1,1,1,1,-1.0,2.0,2.0,2.0,...,1.132406,1.358321,1.143407,-0.460233,-0.754886,-0.503887,-1.143407,1.313355,1.177210,0.701998
4,0,0,1,1,1,1,-1.0,-1.0,-1.0,-1.0,...,-0.416480,1.494382,-0.379260,-0.510947,-2.467059,-0.506512,0.379260,-0.136969,-0.379637,-0.694053


In [40]:
clean_X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5282 entries, 0 to 5281
Data columns (total 92 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Sex                                        5282 non-null   int32  
 1   IsSeniorCitizen                            5282 non-null   int64  
 2   HasPartner                                 5282 non-null   int32  
 3   HasChild                                   5282 non-null   int32  
 4   HasPhoneService                            5282 non-null   int32  
 5   IsBillingPaperless                         5282 non-null   int32  
 6   HelmertEncoder_HasMultiplePhoneNumbers_1   5282 non-null   float64
 7   HelmertEncoder_HasInternetService_1        5282 non-null   float64
 8   HelmertEncoder_HasOnlineSecurityService_1  5282 non-null   float64
 9   HelmertEncoder_HasOnlineBackup_1           5282 non-null   float64
 10  HelmertEncoder_HasDevice

In [43]:
from automl_alex import BestSingleModel, BestSingleModelClassifier

In [51]:
model = BestSingleModelClassifier(
    clean_and_encod_data=True,
    opt_data_prepare=True,
    models_names = ['LightGBM', 'ExtraTrees', 'RandomForest',],
    cat_encoder_names= [
            "HelmertEncoder",
            "OneHotEncoder",
            "CountEncoder",
            "HashingEncoder",
            "BackwardDifferenceEncoder",
        ],
    target_encoders_names= [
            "TargetEncoder",
            "JamesSteinEncoder",
            "CatBoostEncoder",
        ],
    clean_outliers = [True, False],
    num_generator_select_operations = True,
    num_generator_operations = ["/", "*", "-"],
    auto_parameters=True,
    feature_selection=False,
    random_state=42)

12:53:27 | metric is None! Default metric will be used. classifier: AUC, regression: MSE


In [52]:
history = model.opt(X_train, y_train, timeout=2000, verbose=3)

12:53:29 | ##################################################
12:53:30 | > Step 1: calc parameters and pruned score: get test 10 trials
12:54:22 |  One iteration ~ 5.2 sec
12:54:22 |  Possible iters ~ 387.0
12:54:22 | --------------------------------------------------
12:54:22 |   Pruned Threshold Score: 0.8215
12:54:22 | ##################################################
12:54:22 | > Step 2: Full opt with Threshold Score Pruner
12:54:22 | ##################################################
12:54:22 | > Start optimization with the parameters:
12:54:22 | CV_Folds = 7
12:54:22 | Score_CV_Folds = 3
12:54:22 | Feature_Selection = False
12:54:22 | Opt_lvl = 2
12:54:22 | Cold_start = 20
12:54:22 | Early_stoping = 30
12:54:22 | Metric = roc_auc_score
12:54:22 | Direction = maximize
12:54:22 | ##################################################
Optimize: : 196it [16:54,  5.18s/it, | Model: LightGBM | OptScore: 0.8432 | Best roc_auc_score: 0.8439 ]    
13:11:17 | 
 EarlyStopping Exceeded: Best Sc

Finished loading model, total used 500 iterations
Finished loading model, total used 500 iterations
Finished loading model, total used 500 iterations
Finished loading model, total used 500 iterations
Finished loading model, total used 500 iterations
Finished loading model, total used 500 iterations


13:11:24 | Best Score: 0.8439 roc_auc_score


Finished loading model, total used 500 iterations


In [47]:
clean_X_test = de.fit_transform(X_test)

12:50:28 | Source data shape: (1761, 19)
12:50:28 | ##################################################
12:50:28 | ! START preprocessing Data
12:50:28 | > Binary Features
12:50:28 | > Clean Categorical Features
12:50:29 | > Transform Categorical Features.
12:50:29 |  - Encoder: HelmertEncoder ADD features: 21
12:50:29 |  - Encoder: OneHotEncoder ADD features: 31
12:50:29 |   No nans features
12:50:29 | > Start fit DenoisingAutoencoder


Epoch 1/50
Epoch 2/50
Epoch 3/50
Epoch 4/50
Epoch 5/50
Epoch 6/50
Epoch 7/50
Epoch 8/50
Epoch 9/50
Epoch 10/50
Epoch 11/50
Epoch 12/50
Epoch 13/50
Epoch 14/50
Epoch 15/50
Epoch 16/50
Epoch 17/50
Epoch 18/50
Epoch 19/50
Epoch 20/50
Epoch 21/50
Epoch 22/50
Epoch 23/50
Epoch 24/50
Epoch 25/50
Epoch 26/50
Epoch 27/50
Epoch 28/50
Epoch 29/50
Epoch 30/50
Epoch 31/50
Epoch 32/50
Epoch 33/50
Epoch 34/50
Epoch 35/50
Epoch 36/50
Epoch 37/50
Epoch 38/50
Epoch 39/50
Epoch 40/50
Epoch 41/50
Epoch 42/50
Epoch 43/50
Epoch 44/50
Epoch 45/50
Epoch 46/50
Epoch 47/50
Epoch 48/50
Epoch 49/50
Epoch 50/50


12:50:42 | > Add Denoising features
12:50:42 | > CleanOutliers
12:50:42 | > Generate interaction Num Features
12:50:42 |  ADD features: 15
12:50:42 | > Normalization Features
12:50:43 | ##################################################
12:50:43 | Final data shape: (1761, 92)
12:50:43 | Total ADD columns: 73
12:50:43 | Time fit data_preproc: 14.61 sec
12:50:43 | ##################################################


In [53]:
print(model.best_model_name)
print(model.best_model_param)

LightGBM
{'random_seed': 42, 'num_iterations': 500, 'verbose': -1, 'device_type': 'cpu', 'objective': 'binary', 'num_leaves': 2, 'learning_rate': 0.05327604416583107, 'min_child_samples': 2, 'bagging_fraction': 0.8, 'feature_fraction': 0.8, 'bagging_freq': 11}


In [54]:
model.study.best_params

{'*': False,
 '-': False,
 '/': True,
 'BackwardDifferenceEncoder': True,
 'CatBoostEncoder': True,
 'CountEncoder': True,
 'HashingEncoder': True,
 'HelmertEncoder': True,
 'JamesSteinEncoder': False,
 'OneHotEncoder': False,
 'TargetEncoder': False,
 'de_clean_outliers': False,
 'lgbm_bagging_fraction': 0.8,
 'lgbm_bagging_freq': 11,
 'lgbm_feature_fraction': 0.8,
 'lgbm_learning_rate': 0.05327604416583107,
 'lgbm_min_child_samples': 2,
 'lgbm_num_iterations': 500,
 'lgbm_num_leaves': 2,
 'model_name': 'LightGBM'}

In [61]:
predicts = model.predict(clean_X_test)
predicts

ValueError: columns overlap but no suffix specified: Index(['HelmertEncoder_HasMultiplePhoneNumbers_1',
       'HelmertEncoder_HasInternetService_1',
       'HelmertEncoder_HasOnlineSecurityService_1',
       'HelmertEncoder_HasOnlineBackup_1',
       'HelmertEncoder_HasDeviceProtection_1',
       'HelmertEncoder_HasTechSupportAccess_1', 'HelmertEncoder_HasOnlineTV_1',
       'HelmertEncoder_HasMovieSubscription_1',
       'HelmertEncoder_HasContractPhone_1', 'HelmertEncoder_PaymentMethod_2',
       'HelmertEncoder_HasMultiplePhoneNumbers_0',
       'HelmertEncoder_HasInternetService_0',
       'HelmertEncoder_HasOnlineSecurityService_0',
       'HelmertEncoder_HasOnlineBackup_0',
       'HelmertEncoder_HasDeviceProtection_0',
       'HelmertEncoder_HasTechSupportAccess_0', 'HelmertEncoder_HasOnlineTV_0',
       'HelmertEncoder_HasMovieSubscription_0',
       'HelmertEncoder_HasContractPhone_0', 'HelmertEncoder_PaymentMethod_0',
       'HelmertEncoder_PaymentMethod_1'],
      dtype='ob

### Самый простой вариант, с ручным разделение на train/val

In [9]:
# Разделим на train|val
data_train, data_val, target_train, target_val = train_test_split(data[feature_cols], data[target_col], 
                                                    train_size=0.8,
                                                    random_state=42)

In [ ]:
# Проведем one-hot кодирование
def data_encoding(data):
    dummy_features = pd.get_dummies(data[cat_cols], drop_first=True)
    data = pd.concat([data[num_cols], dummy_features], axis=1)
    return data

In [ ]:
data_train = data_encoding(data_train)

In [ ]:
data_train.head()

In [ ]:
data_val = data_encoding(data_val)

In [ ]:
# Сделаем нормировку в Pandas с сохранением среднего и стандартного отклонения для использования в будущем для теста
scaling_dict = {}
for column in num_cols:
    mean = data_train[column].mean()
    dev = data_train[column].std()
    scaling_dict[column + '_mean'] = mean
    scaling_dict[column + '_dev'] = dev
    data_train[column] = (data_train[column] - mean)/dev

In [ ]:
# Теперь нормировку применим к валидационной выборке
for column in num_cols:
    data_val[column] = (data_val[column] - scaling_dict[column + '_mean'])/scaling_dict[column + '_dev']

In [ ]:
data_val.head()

### Логистическая регрессия с ручным подбором параметров


In [ ]:
model_log_reg = LogisticRegressionCV(
    scoring="roc_auc", refit=True, random_state=0, max_iter=500
).fit(data_train, target_train)

In [ ]:
model_log_reg.score(data_train, target_train)

In [ ]:
# а что на валидационном сете?
model_log_reg.score(data_val, target_val)

In [ ]:
model_log_reg.predict(data_val)

In [ ]:
model_log_reg.get_params()

In [ ]:
# добавим еще информацию об неоднородности классов
class_weight_0 = 1 - target_train.mean()
class_weight_0

In [ ]:
# попробую еще L1 вместо L2
model_log_reg = LogisticRegressionCV(
    scoring="roc_auc",
    refit=True,
    random_state=0,
    max_iter=1000,
    penalty="l1",
    solver="saga",
    class_weight={0: class_weight_0, 1: 1 - class_weight_0},
).fit(data_train, target_train)

In [ ]:
model_log_reg.score(data_val, target_val)

In [ ]:
# снова L2 но с классами
model_log_reg = LogisticRegressionCV(
    scoring="roc_auc",
    refit=True,
    random_state=0,
    max_iter=1000,
    class_weight={0: class_weight_0, 1: 1 - class_weight_0},
).fit(data_train, target_train)
model_log_reg.score(data_val, target_val)

In [ ]:
model_log_reg.get_params()

ОК, обучаем на полном датасете с лучшими параметрами, L2 и классами

In [ ]:
data_train = data_encoding(data[feature_cols])
scaling_dict = {}
for column in num_cols:
    mean = data_train[column].mean()
    dev = data_train[column].std()
    scaling_dict[column + '_mean'] = mean
    scaling_dict[column + '_dev'] = dev
    data_train[column] = (data_train[column] - mean)/dev

In [ ]:
model_log_reg = LogisticRegressionCV(
    Cs=[10],
    scoring="roc_auc",
    refit=True,
    random_state=0,
    max_iter=1000,
    class_weight={0: class_weight_0, 1: 1 - class_weight_0},
).fit(data_train, data[target_col])
model_log_reg.score(data_train, data[target_col])

#### Результат на Kaggle 0.84568

### Теперь логистическую регрессию, но средствами sklearn полностью с разным кодированием

In [ ]:
data_train, data_val, target_train, target_val = train_test_split(data[feature_cols], data[target_col], 
                                                    train_size=0.8,
                                                    random_state=42)

In [ ]:
# One-hot
numeric_features = num_cols
numeric_transformer = Pipeline(steps=[("scaler", StandardScaler())])

categorical_features = cat_cols
categorical_transformer = OneHotEncoder(drop="if_binary")

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

model_lr = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        (
            "classifier",
            LogisticRegressionCV(
                Cs=[10],
                scoring="roc_auc",
                refit=True,
                random_state=0,
                max_iter=1000,
                class_weight={0: class_weight_0, 1: 1 - class_weight_0},
            ),
        ),
    ]
)

In [ ]:
model_lr.fit(data_train, target_train)
model_lr.predict(data_val)
model_lr.score(data_val, target_val)

In [ ]:
# Target encoding
numeric_features = num_cols
numeric_transformer = Pipeline(steps=[("scaler", StandardScaler())])

categorical_features = cat_cols
categorical_transformer = TargetEncoder()

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

model_lr = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        (
            "classifier",
            LogisticRegressionCV(
                Cs=[10],
                scoring="roc_auc",
                refit=True,
                random_state=0,
                max_iter=1000,
                class_weight={0: class_weight_0, 1: 1 - class_weight_0},
            ),
        ),
    ]
)

In [ ]:
model_lr.fit(data_train, target_train)
model_lr.predict(data_val)
model_lr.score(data_val, target_val)

In [ ]:
# LOO encoding
numeric_features = num_cols
numeric_transformer = Pipeline(steps=[("scaler", StandardScaler())])

categorical_features = cat_cols
categorical_transformer = LeaveOneOutEncoder()

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

model_lr = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        (
            "classifier",
            LogisticRegressionCV(
                Cs=[10],
                scoring="roc_auc",
                refit=True,
                random_state=0,
                max_iter=1000,
                class_weight={0: class_weight_0, 1: 1 - class_weight_0},
            ),
        ),
    ]
)

In [ ]:
model_lr.fit(data_train, target_train)
model_lr.predict(data_val)
model_lr.score(data_val, target_val)

In [ ]:
# BackwardDifferenceEncoder
numeric_features = num_cols
numeric_transformer = Pipeline(steps=[("scaler", StandardScaler())])

categorical_features = cat_cols
categorical_transformer = BackwardDifferenceEncoder()

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

model_lr = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        (
            "classifier",
            LogisticRegressionCV(
                Cs=[10],
                scoring="roc_auc",
                refit=True,
                random_state=0,
                max_iter=1000,
                class_weight={0: class_weight_0, 1: 1 - class_weight_0},
            ),
        ),
    ]
)

In [ ]:
model_lr.fit(data_train, target_train)
model_lr.predict(data_val)
model_lr.score(data_val, target_val)

С незначительной разницей победил обычный backward difference, на нем и обучим финальную модель

In [ ]:
model_lr.fit(data[feature_cols], data[target_col])

#### Результат на Kaggle 0.84571

### Попробую Random Forest

In [ ]:
data_train, data_val, target_train, target_val = train_test_split(data[feature_cols], data[target_col], 
                                                    train_size=0.8,
                                                    random_state=42)
data_train = data_encoding(data_train)

In [ ]:
data_train.head()

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from tqdm.notebook import tqdm

In [ ]:
model = RandomForestClassifier(n_estimators=20, n_jobs=-1, max_depth = 10)

model.fit(data_train, target_train)
target_train_predicted = model.predict_proba(data_train)[:, 1]
train_auc = roc_auc_score(target_train, target_train_predicted)
train_auc

In [ ]:
data_val = data_encoding(data_val)

In [ ]:
target_val_predicted = model.predict_proba(data_val)[:, 1]
val_auc = roc_auc_score(target_val, target_val_predicted)
val_auc

In [ ]:
model_rf = RandomForestClassifier(random_state=42)
param_grid = { 
    'n_estimators': [10, 20, 50, 100],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,6,8,10,12],
    'criterion' :['gini']
}

In [ ]:
cv_rf = GridSearchCV(estimator=model_rf, param_grid=param_grid, cv=5)
cv_rf.fit(data_train, target_train)
cv_rf.best_params_

In [ ]:
model_rf=RandomForestClassifier(random_state=42, max_features='auto', n_estimators= 50, max_depth=8, criterion='gini')

model_rf.fit(data_train, target_train)
target_val_predicted = model_rf.predict_proba(data_val)[:, 1]
val_auc = roc_auc_score(target_val, target_val_predicted)
val_auc

Выпишите какое лучшее качество и с какими параметрами вам удалось получить

## Применение градиентного бустинга (2 балла)

Если вы хотите получить баллы за точный ответ, то стоит попробовать градиентный бустинг. Часто градиентный бустинг с дефолтными параметрами даст вам 80% результата за 0% усилий.

Мы будем использовать catboost, поэтому нам не надо кодировать категориальные признаки. catboost сделает это сам (в .fit() надо передать cat_features=cat_cols). А численные признаки нормировать для моделей, основанных на деревьях не нужно.

1) Разделите выборку на train/valid. Протестируйте catboost cо стандартными параметрами.

2) Протестируйте разные занчения параметроа количества деревьев и learning_rate'а и выберите лучшую по метрике ROC-AUC комбинацию. 

(Дополнительно) Есть некоторые сложности с тем, чтобы использовать CatBoostClassifier вместе с GridSearchCV, поэтому мы не просим использовать кроссвалидацию. Но можете попробовать)

In [ ]:
from catboost import CatBoostClassifier, Pool
from catboost.utils import eval_metric
from catboost.utils import get_roc_curve
from sklearn.metrics import auc

In [ ]:
data_train, data_val, target_train, target_val = train_test_split(data[feature_cols], data[target_col], 
                                                    train_size=0.8,
                                                    random_state=42)
#data_train = data_encoding(data_train)
#data_val = data_encoding(data_val)

In [ ]:
test_data = Pool(data_val, target_val, cat_features=cat_cols)

In [ ]:
# попробуем искать с помощью встроенного гридсерча на всем датасете

grid = {
    "learning_rate": [0.03, 0.1, 0.5, 1],
    "depth": [1, 2, 4, 6, 10],
    "l2_leaf_reg": [1, 3, 5, 7, 9],
    "iterations": [8, 10, 12, 20, 24, 30],
}

model_cat = CatBoostClassifier(
    loss_function="Logloss",
    verbose=False,
    eval_metric="AUC",
    cat_features=cat_cols
)
grid_search_result = model_cat.grid_search(
    grid, data[feature_cols], data[target_col], plot=True
)

In [ ]:
class_weight_0 = 1 - data['Churn'].mean()
class_weight_0

In [ ]:
# обучим с лучшими параметрами и весами классов
model_cat = CatBoostClassifier(
                iterations=30,
                depth=4,
                learning_rate=1,
                loss_function="Logloss",
                l2_leaf_reg = 7,
                verbose=True,
                eval_metric="AUC",
                class_weights= [class_weight_0, 1-class_weight_0]
            )

model_cat.fit(data[feature_cols], data[target_col], cat_features=cat_cols);

#### К вопросу о воспроизводимости результатов, я как-то выбил 0.85169 на каггле, но не помню с какими параметрами, первый перебор параметров проводил руками в цикле. С указанными выше 0,85 не получается 

### Попробую LightGBM

In [10]:
import lightgbm as lgb

In [11]:
data_train.head()

,ClientPeriod,MonthlySpending,TotalSpent,Sex,IsSeniorCitizen,HasPartner,HasChild,HasPhoneService,HasMultiplePhoneNumbers,HasInternetService,HasOnlineSecurityService,HasOnlineBackup,HasDeviceProtection,HasTechSupportAccess,HasOnlineTV,HasMovieSubscription,HasContractPhone,IsBillingPaperless,PaymentMethod
4556,31,73.55,2094.65,Female,1,No,No,Yes,No,Fiber optic,Yes,No,No,No,No,No,Month-to-month,Yes,Credit card (automatic)
1468,59,100.05,6034.85,Female,0,Yes,No,Yes,No,Fiber optic,No,Yes,No,Yes,Yes,Yes,Two year,Yes,Mailed check
3142,60,99.00,6017.90,Male,1,Yes,No,Yes,Yes,Fiber optic,No,No,No,Yes,Yes,Yes,Month-to-month,Yes,Electronic check
2093,6,73.85,401.30,Female,0,No,No,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Mailed check
1505,72,24.75,1859.10,Male,0,Yes,Yes,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Credit card (automatic)


In [12]:
for c in data_train.columns:
    col_type = data_train[c].dtype
    if col_type == 'object' or col_type.name == 'category':
        data_train[c] = data_train[c].astype('category')

In [13]:
for c in data_val.columns:
    col_type = data_val[c].dtype
    if col_type == 'object' or col_type.name == 'category':
        data_val[c] = data_val[c].astype('category')

In [14]:
data_val.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1057 entries, 1448 to 274
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   ClientPeriod              1057 non-null   int64   
 1   MonthlySpending           1057 non-null   float64 
 2   TotalSpent                1057 non-null   float64 
 3   Sex                       1057 non-null   category
 4   IsSeniorCitizen           1057 non-null   int64   
 5   HasPartner                1057 non-null   category
 6   HasChild                  1057 non-null   category
 7   HasPhoneService           1057 non-null   category
 8   HasMultiplePhoneNumbers   1057 non-null   category
 9   HasInternetService        1057 non-null   category
 10  HasOnlineSecurityService  1057 non-null   category
 11  HasOnlineBackup           1057 non-null   category
 12  HasDeviceProtection       1057 non-null   category
 13  HasTechSupportAccess      1057 non-null   cate

In [15]:
fit_params={"early_stopping_rounds":100, 
            "eval_metric" : 'auc', 
            "eval_set" : [(data_val, target_val)],
            'eval_names': ['valid'],
            'verbose': 100,
            'feature_name': 'auto', # that's actually the default
            'categorical_feature': 'auto' # that's actually the default
           }

In [16]:
model_lgb = lgb.LGBMClassifier(num_leaves= 15, max_depth=-1, 
                         random_state=314, 
                         silent=True, 
                         metric='None', 
                         n_jobs=4, 
                         n_estimators=1000,
                         colsample_bytree=0.9,
                         subsample=0.9,
                         learning_rate=0.1)

In [ ]:
model_lgb.fit(data_train, target_train, **fit_params)

In [18]:
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from lightgbm import LGBMClassifier

In [19]:
model_lgb = LGBMClassifier()
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model_lgb, data_train, target_train, scoring='roc_auc', cv=cv, n_jobs=-1)
print('AUC: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))

AUC: 0.834 (0.020)


In [20]:
def get_models():
    models = dict()
    trees = [3, 5, 10, 20, 30]
    for n in trees:
        models[str(n)] = LGBMClassifier(n_estimators=n)
    return models

In [21]:
def evaluate_model(model):
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    scores = cross_val_score(model, data_train, target_train, scoring='roc_auc', cv=cv, n_jobs=-1)
    return scores

In [22]:
models = get_models()
results, names = list(), list()
for name, model in models.items():
    scores = evaluate_model(model)
    results.append(scores)
    names.append(name)
    print('>%s %.3f (%.3f)' % (name, np.mean(scores), np.std(scores)))

>3 0.838 (0.020)
>5 0.840 (0.021)
>10 0.843 (0.021)
>20 0.844 (0.020)
>30 0.844 (0.021)


In [23]:
def get_models():
    models = dict()
    for i in range(1,11):
        models[str(i)] = LGBMClassifier(max_depth=i, num_leaves=2**i, n_estimators=20)
    return models

In [24]:
models = get_models()
results, names = list(), list()
for name, model in models.items():
    scores = evaluate_model(model)
    results.append(scores)
    names.append(name)
    print('>%s %.3f (%.3f)' % (name, np.mean(scores), np.std(scores)))

>1 0.839 (0.022)
>2 0.846 (0.021)
>3 0.849 (0.021)
>4 0.849 (0.020)
>5 0.846 (0.020)
>6 0.842 (0.020)
>7 0.838 (0.019)
>8 0.835 (0.020)
>9 0.834 (0.020)
>10 0.832 (0.020)


In [25]:
model_lgb = lgb.LGBMClassifier(
    num_leaves=2 ** 4,
    max_depth=4,
    random_state=314,
    silent=True,
    metric="None",
    learning_rate=0.1,
    n_estimators=20,
)

In [36]:
for c in data.columns:
    col_type = data[c].dtype
    if col_type == 'object' or col_type.name == 'category':
        data[c] = data[c].astype('category')

In [46]:
model_lgb = LGBMClassifier(max_depth=4, num_leaves=2**4, n_estimators=20)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model_lgb, data[feature_cols], data[target_col], scoring='roc_auc', cv=cv, n_jobs=-1)
print('AUC: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))

AUC: 0.845 (0.018)


In [49]:
model_lgb.fit(data[feature_cols], data[target_col], **fit_params)

Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[20]	valid's auc: 0.846155	valid's binary_logloss: 0.420643


LGBMClassifier(max_depth=4, n_estimators=20, num_leaves=16)

0.84890 на тестовых данных

# Предсказания

In [50]:
best_model = model_lgb

In [39]:
X_test = pd.read_csv('./test.csv')
submission = pd.read_csv('./submission.csv')

In [40]:
data_cleaning(X_test)

In [41]:
# шаг только для LightGBM
for c in X_test.columns:
    col_type = X_test[c].dtype
    if col_type == 'object' or col_type.name == 'category':
        X_test[c] = X_test[c].astype('category')

In [ ]:
# этот шаг применим не для всех моделей!
X_test = data_encoding(X_test)

In [ ]:
# этот шаг применим не для всех моделей!
for column in num_cols:
    X_test[column] = (X_test[column] - scaling_dict[column + '_mean'])/scaling_dict[column + '_dev']

In [42]:
X_test.head()

,ClientPeriod,MonthlySpending,TotalSpent,Sex,IsSeniorCitizen,HasPartner,HasChild,HasPhoneService,HasMultiplePhoneNumbers,HasInternetService,HasOnlineSecurityService,HasOnlineBackup,HasDeviceProtection,HasTechSupportAccess,HasOnlineTV,HasMovieSubscription,HasContractPhone,IsBillingPaperless,PaymentMethod
0,42,56.10,2386.85,Male,0,Yes,No,Yes,Yes,DSL,Yes,No,No,No,No,No,One year,No,Credit card (automatic)
1,29,98.50,3004.15,Male,1,Yes,No,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check
2,9,76.25,684.85,Male,0,No,No,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check
3,67,74.00,4868.40,Female,0,Yes,Yes,Yes,Yes,DSL,Yes,Yes,No,Yes,Yes,No,Two year,No,Credit card (automatic)
4,71,20.10,1389.60,Female,0,Yes,Yes,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Bank transfer (automatic)


In [51]:
submission['Churn'] = 1 - best_model.predict_proba(X_test)
submission.to_csv('./my_submission.csv', index=False)

# Kaggle (5 баллов)

Как выставить баллы:

1) 1 >= roc auc > 0.84 это 5 баллов

2) 0.84 >= roc auc > 0.7 это 3 балла

3) 0.7 >= roc auc > 0.6 это 1 балл

4) 0.6 >= roc auc это 0 баллов


Для выполнения задания необходимо выполнить следующие шаги.
* Зарегистрироваться на платформе [kaggle.com](kaggle.com). Процесс выставления оценок будет проходить при подведении итогового рейтинга. Пожалуйста, укажите во вкладке Team -> Team name свои имя и фамилию в формате Имя_Фамилия (важно, чтобы имя и фамилия совпадали с данными на Stepik).
* Обучить модель, получить файл с ответами в формате .csv и сдать его в конкурс. Пробуйте и экспериментируйте. Обратите внимание, что вы можете выполнять до 20 попыток сдачи на kaggle в день.
* После окончания соревнования отправить в итоговый ноутбук с решением на степик. 
* После дедлайна проверьте посылки других участников по критериям. Для этого надо зайти на степик, скачать их ноутбук и проверить скор в соревновании.